In [ ]:
# imports
import os
import requests
import pandas as pd
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# Load API key 
FOURSQUARE_API_KEY = os.getenv('test_project2')

def get_venues_fs(latitude, longitude, radius, categories):
  
  url = 'https://api.foursquare.com/v3/places/search'

  headers = {
    'accept': 'application/json',
    'Authorization': FOURSQUARE_API_KEY    
  }

  params = {
    'll': f'{latitude},{longitude}',
    'radius': radius,
    'limit': 50, 
    'categories' : ','.join(categories)
  }
  
  response = requests.get(url, 
                       params=params,
                       headers=headers)

  return response.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Define the list of categories (e.g., '16032' category ID for Parks)
categories = ['16032']

# Initialize a list to store venue data
venue_list = []

# Iterate through the DataFrame rows
for index, row in bikes_df.iterrows():
    lat = row['lat']
    lon = row['lon']

    # Call API using the provided latitude, longitude, radius, and categories
    data = get_venues_fs(lat, lon, 1500, categories)

    # Append the number of venues retrieved to the venue_list
    venue_list.append({
        'lat': lat,
        'lon': lon,
        'fs_parks': len(data['results'])
    })

Put your parsed results into a DataFrame

In [ ]:
# Create a DataFrame from the venue data list
foursquare_parks_df = pd.DataFrame(venue_list)

foursquare_parks_df.to_csv('foursquare_parks_df.csv')

# Display the resulting DataFrame
print(foursquare_parks_df.head(40))

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# Load API key
YELP_API_KEY = os.getenv('Yelp_key')


def get_parks_yelp(latitude, longitude, radius):
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
        'Authorization': f'Bearer {YELP_API_KEY}'
    }

    params = {
        'latitude': lat,
        'longitude': lon,
        'radius': radius,
        'categories': 'parks', 
        'limit': 50  # Limit the number of results
    }

    response = requests.get(url, headers=headers, params=params)
    
    return response.json()


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Create an empty list to store Yelp venue data
yelp_venue_data = []

# Assuming bikes_df is defined as before
for index, row in bikes_df.head(500).iterrows():
    lat = row['lat']
    lon = row['lon']

    # Call Yelp API for parks using the provided latitude, longitude, and radius
    data = get_parks_yelp(lat, lon, 1500)
    
    # Print the data dictionary to inspect its structure
    print(data)
    
    # Append park venue count and location information to the yelp_venue_data list
    yelp_venue_data.append({'lat': lat, 'lon': lon, 'yelp_parks': len(data['businesses'])})


Put your parsed results into a DataFrame

In [ ]:
# Create a DataFrame from the yelp_venue_data list
yelp_parks_df = pd.DataFrame(yelp_venue_data)

# Display the yelp_venue_df DataFrame
print(yelp_parks_df.head(40))

#Safe yelp_parks_df as a csv file
yelp_parks_df.to_csv('yelp_parks_df.csv')

In [ ]:
# Merge Foursquare and Yelp data as venue DataFrame
Venue_df = foursquare_parks_df.merge(yelp_parks_df, on=['lat','lon'])
venue_df.head(10)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [ ]:
# Calculate the average park counts from Foursquare and Yelp
average_fs_parks = venue_df['fs_parks'].mean()
average_yelp_parks = venue_df['yelp_parks'].mean()

# Compare the average park counts
if average_fs_parks > average_yelp_parks:
    better_api = 'Foursquare'
else:
    better_api = 'Yelp'

print(f"The API with better coverage in terms of average park counts is {better_api}.")

# Sum the counts of Foursquare parks and Yelp parks for each API
total_fs_parks = venue_df['fs_parks'].sum()
total_yelp_parks = venue_df['yelp_parks'].sum()

# Compare the total counts and determine which API has better coverage
if total_fs_parks > total_yelp_parks:
    better_coverage_api = 'Foursquare'
else:
    better_coverage_api = 'Yelp'

print(f"The API with better coverage in terms of parks is: {better_coverage_api}")


Foursquare has better coverage both in terms of average park counts and total park counts. This is determined by comparing the calculated values for Foursquare and Yelp and making decisions based on these comparisons.

Get the top 10 restaurants according to their rating

In [ ]:
# Load Yelp API key
YELP_API_KEY = os.getenv('Yelp_key')

# Define the API endpoint for business search
url = 'https://api.yelp.com/v3/businesses/search'

# Define parameters for the request
params = {
    'latitude': lat,  # Use the latitude of the bike station
    'longitude': lon, # Use the longitude of the bike station
    'radius': 1500,    # Set the search radius
    'categories': 'restaurants',  # Limit to restaurants
    'limit': 10,       # Get top 10 restaurants
    'sort_by': 'rating'  # Sort by rating
}

# Set the authorization header
headers = {
    'Authorization': f'Bearer {YELP_API_KEY}'
}

# Make the API request
response = requests.get(url, headers=headers, params=params)
data = response.json()

# Display the top 10 restaurants
top_restaurants = data['businesses']
for idx, restaurant in enumerate(top_restaurants, start=1):
    print(f"{idx}. {restaurant['name']} - Rating: {restaurant['rating']}")

Output:
1. Good Vibes Kitchen - Rating: 5.0
2. Ballissimo Loukoumades Bar - Rating: 5.0
3. The MoMo House - Rating: 5.0
4. Galata Restaurant - Rating: 5.0
5. Papyrus - Rating: 5.0
6. Trattoria Di Parma - Rating: 5.0
7. Bancan Kitchen - Rating: 4.5
8. Hanamaru - Rating: 4.5
9. Momo Hut & Gardens - Rating: 4.5
10. Trinacria Espresso - Rating: 4.5